# Notebook on research put-call parity

In [118]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
pd.set_option('display.max_columns', 500)
import seaborn as sns
import plotly.graph_objects as go
import datetime
from utils import get_human_timestamp
import plotly
import plotly.graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots


In [2]:
df = pd.read_csv(f"./datasets/deribit_options_chain_2022-12-01_OPTIONS.csv")
data = df.copy()

## Отбираем нужные колонки и опционы только на одну криптовалюту

In [91]:
def preprocess_data(df: pd.DataFrame, currency: str  = "BTC"):
    ind = [symbol.startswith(currency) for symbol in df.symbol]
    needed_columns = ["timestamp", "type", "strike_price", "expiration", "mark_price", "underlying_price"]
    data = df[ind].reset_index(drop=True)[needed_columns]
    data["human_expiration"] = data["expiration"].apply(get_human_timestamp)
    data["human_timestamp"] = data["timestamp"].apply(get_human_timestamp)
    return data

In [92]:
single_currency_chain = preprocess_data(data)

In [93]:
single_currency_chain

,timestamp,type,strike_price,expiration,mark_price,underlying_price,human_expiration,human_timestamp
0,1669852800273000,put,17250,1669881600000000,0.0095,17165.5125,2022-12-01 11:00:00.000000,2022-12-01 03:00:00.273000
1,1669852800273000,call,17500,1671177600000000,0.0377,17162.3600,2022-12-16 11:00:00.000000,2022-12-01 03:00:00.273000
2,1669852800302000,call,16500,1669881600000000,0.0393,17165.5125,2022-12-01 11:00:00.000000,2022-12-01 03:00:00.302000
3,1669852800324000,call,16500,1669881600000000,0.0393,17165.5125,2022-12-01 11:00:00.000000,2022-12-01 03:00:00.324000
4,1669852800402000,put,250000,1672387200000000,13.5643,17164.6600,2022-12-30 11:00:00.000000,2022-12-01 03:00:00.402000
...,...,...,...,...,...,...,...,...
10432075,1669939199983000,call,34000,1672387200000000,0.0004,16953.1600,2022-12-30 11:00:00.000000,2022-12-02 02:59:59.983000
10432076,1669939199983000,call,10000,1695974400000000,0.4728,16962.0900,2023-09-29 11:00:00.000000,2022-12-02 02:59:59.983000
10432077,1669939199984000,call,5000,1680249600000000,0.7089,16946.2200,2023-03-31 11:00:00.000000,2022-12-02 02:59:59.984000
10432078,1669939199988000,call,13500,1670572800000000,0.2059,16975.1700,2022-12-09 11:00:00.000000,2022-12-02 02:59:59.988000


In [153]:
def plot_single_expiration(df: pd.DataFrame, strike_given: int = False, expiration_date_given: int = False) -> None:
    # select options with similar expiration date
    single_expiration_date = df.sample(1).iloc[0]["expiration"]
    single_expiration_date_human = get_human_timestamp(single_expiration_date)
    single_expiration_date = expiration_date_given if expiration_date_given else single_expiration_date
    single_expiration_df = df[df["expiration"] == single_expiration_date].copy()
    
    # select options with similar strikes
    single_strike = single_expiration_df.sample(1).iloc[0]["strike_price"]
    single_strike = strike_given if strike_given else single_strike
    single_strike_df = single_expiration_df[single_expiration_df["strike_price"] == single_strike].copy()
    
    #divide by calls and puts and merge
    calls = single_strike_df[single_strike_df["type"] == "call"]
    calls["call_price_usd"] = calls["underlying_price"]*calls["mark_price"]
    puts = single_strike_df[single_strike_df["type"] == "put"]
    puts["put_price_usd"] = puts["underlying_price"]*puts["mark_price"]
    calls = calls.rename(columns = {"mark_price": "call_price"})[["timestamp", "call_price", 
        "strike_price", "underlying_price", "human_timestamp", "call_price_usd"]].copy()
    puts = puts.rename(columns = {"mark_price": "put_price"})[["timestamp", "put_price", "put_price_usd"]].copy()
    result = calls.merge(puts, on = "timestamp")
    result["parity"] = result["underlying_price"] - result["call_price_usd"] + result["put_price_usd"]
    
    # plotting
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=result["human_timestamp"], y=result["call_price_usd"], name='call price, usd'))
    fig.add_trace(go.Scatter(x=result["human_timestamp"], y=result["put_price_usd"], name='put price, usd'))
    fig.add_trace(go.Scatter(x=result["human_timestamp"], y=result["underlying_price"], name='underlying price, usd'))
    fig.add_trace(go.Scatter(x=result["human_timestamp"], y=result["parity"], name='parity'))
    # fig.add_trace(go.Scatter(x=result["human_timestamp"], y=result["strike_price"], name='strike'))
    fig.update_layout(title=f"Strike: {single_strike} USD, expiration: {single_expiration_date_human}")
    fig.show()
    
    return result


In [154]:
final_df = plot_single_expiration(single_currency_chain)

In [155]:
# fixed date and strike
final_df = plot_single_expiration(single_currency_chain, strike_given=10000, expiration_date_given = 1695974400000000)